# PlayerxChampion Trueskill

### Helpful Resources
[Trueskill Documentation](https://trueskill.org/#version-0-4-4)  
[Trueskill for DOTA2 Players - Kaggle](https://www.kaggle.com/devinanzelmo/dota-2-skill-rating-with-trueskill)  
[Collection of Trueskill Examples](https://python.hotexamples.com/examples/trueskill/TrueSkill/-/python-trueskill-class-examples.html)  

In [1]:
# Housekeeping
import requests
import datetime
import pandas as pd
import numpy as np
import io
import os
import trueskill
import itertools
import math
import json
from sklearn.metrics import  f1_score, log_loss, precision_score, recall_score

import oeutils

pd.options.display.max_rows = 100
pd.options.display.max_columns = 500

workingdir = '../data/'
years = ['2018', '2019', '2020', '2021']

#oeutils.download(workingdir, years)
df = oeutils.read(workingdir, years)
df = oeutils.clean(df, split_on='player', keep_identities=False, keep_leagues=False, keep_columns=False)

/home/paulperrone/source/lol-analysis/notebooks/oeutils.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oe_data['gameid'] = oe_data['gameid'].str.strip()


In [2]:
df.shape

(188865, 117)

In [3]:
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
348,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,1,Blue,top,Duke,Invictus Gaming,Ornn,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,3,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9631,363.4340,0.150020,315.4340,NaN,13,0.4906,6,0.2264,2,NaN,NaN,10791,7272,274.4151,0.188550,8870.0,NaN,218.0,216.0,2,1,0,8.2264,3292.0,4758.0,90.0,3287.0,4640.0,88.0,5.0,118.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5280.0,7775.0,150.0,5060.0,7477.0,144.0,220.0,298.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
349,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,2,Blue,jng,Ning,Invictus Gaming,Kha'Zix,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,0,5,12,4,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6924,261.2830,0.107854,693.8113,NaN,48,1.8113,24,0.9057,10,NaN,NaN,10831,7312,275.9245,0.189587,9775.0,NaN,194.0,36.0,158,96,8,7.3208,3170.0,3745.0,67.0,2988.0,3386.0,58.0,182.0,359.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4660.0,5588.0,100.0,4290.0,4957.0,80.0,370.0,631.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
350,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,3,Blue,mid,Rookie,Invictus Gaming,Orianna,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,8,12,4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19985,754.1509,0.311303,274.6415,NaN,10,0.3774,7,0.2642,5,NaN,NaN,12165,8646,326.2642,0.224176,9310.0,NaN,266.0,246.0,20,16,4,10.0377,3633.0,5236.0,108.0,3378.0,5108.0,92.0,255.0,128.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,5389.0,7541.0,157.0,5211.0,8079.0,150.0,178.0,-538.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
351,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,4,Blue,bot,JackeyLove,Invictus Gaming,Ezreal,Azir,Malzahar,Camille,Illaoi,Vladimir,1590,1,3,1,7,12,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.4528,0.6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23623,891.4340,0.367971,321.2075,NaN,14,0.5283,6,0.2264,2,NaN,NaN,14028,10509,396.5660,0.272482,13075.0,NaN,276.0,257.0,19,8,11,10.4151,3992.0,3286.0,97.0,3438.0,3018.0,88.0,554.0,268.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,6800.0,5569.0,159.0,5138.0,5263.0,134.0,1662.0,306.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
352,2899-3157,complete,https://lpl.qq.com/es/stats.shtml?bmid=2899,LPL,2018,Spring,0,2018-01-15 09:40:00,1,8.01,5,Blue,sup,Baolan,Invictus Gaming,Tahm Kench,A

In [4]:
lcs = df[df['league'].isin(['LCS', 'NA LCS'])]
lcs.shape

(7590, 117)

In [5]:
lcs = lcs.sort_values(by=['date', 'player'])

In [6]:
lcs.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
1699,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,8,Red,mid,Bjergsen,TSM,Ryze,Kog'Maw,Tahm Kench,Ezreal,Braum,Shen,1687,0,1,0,0,1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0356,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,8313,295.6609,0.301534,325.6787,406.0225,13,0.4624,5,0.1778,2,24.0,0.8536,10993,7276,258.7789,0.255883,10750.0,NaN,301.0,293.0,8,8,0,10.7054,3620.0,5005.0,103.0,3818.0,4866.0,95.0,-198.0,139.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,5540.0,8006.0,162.0,5897.0,7708.0,146.0,-357.0,298.0,16.0,0.0,0.0,0.0,2.0,0.0,0.0
1695,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,4,Blue,bot,Doublelift,Team Liquid,Tristana,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,5,0,5,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11534,410.2193,0.251691,168.4410,361.2448,14,0.4979,11,0.3912,3,40.0,1.4226,14046,10329,367.3622,0.253797,12375.0,NaN,289.0,253.0,36,13,8,10.2786,3510.0,3360.0,94.0,3563.0,3173.0,97.0,-53.0,187.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,6213.0,5507.0,142.0,5395.0,5280.0,155.0,818.0,227.0,-13.0,2.0,1.0,0.0,0.0,0.0,1.0
1697,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,6,Red,top,Hauntzer,TSM,Gnar,Kog'Maw,Tahm Kench,Ezreal,Braum,Shen,1687,0,0,3,0,1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0356,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,7391,262.8690,0.268091,576.0640,422.3829,15,0.5335,1,0.0356,3,26.0,0.9247,10105,6388,227.1962,0.224653,9075.0,NaN,271.0,271.0,0,0,0,9.6384,3119.0,4254.0,78.0,3650.0,4637.0,77.0,-531.0,-383.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,5087.0,7378.0,140.0,5824.0,7885.0,130.0,-737.0,-507.0,10.0,0.0,0.0,1.0,0.0,3.0,0.0
1692,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,1,Blue,top,Impact,Team Liquid,Gangplank,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,3,0,6,11,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,16858,599.5732,0.367870,386.8880,319.9881,14,0.4979,2,0.0711,2,24.0,0.8536,13474,9757,347.0184,0.239742,12133.0,NaN,254.0,248.0,6,1,4,9.0338,3650.0,4637.0,77.0,3119.0,4254.0,78.0,531.0,383.0,-1.0,0.0,1.0,0.0,0.0,0.0,1.0,5824.0,7885.0,130.0,5087.0,7378.0,140.0,737.0,507.0,-10.0,0.0,3.0,0.0,0.0,0.0,1.0
1698,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,20

In [7]:
input_data = lcs[['gameid', 'date', 'year', 'patch', 'league', 'side', 'team', 'player', 'position', 'champion', 'result']]
input_data.head()

,gameid,date,year,patch,league,side,team,player,position,champion,result
1699,TRLH1/1002440062,2018-01-20 20:50:09,2018,8.01,NA LCS,Red,TSM,Bjergsen,mid,Ryze,0
1695,TRLH1/1002440062,2018-01-20 20:50:09,2018,8.01,NA LCS,Blue,Team Liquid,Doublelift,bot,Tristana,1
1697,TRLH1/1002440062,2018-01-20 20:50:09,2018,8.01,NA LCS,Red,TSM,Hauntzer,top,Gnar,0
1692,TRLH1/1002440062,2018-01-20 20:50:09,2018,8.01,NA LCS,Blue,Team Liquid,Impact,top,Gangplank,1
1698,TRLH1/1002440062,2018-01-20 20:50:09,2018,8.01,NA LCS,Red,TSM,MikeYeung,jng,Shyvana,0


In [8]:
columns = ['gameid', 'date', 'year', 'patch', 'league', \
           'blue_team', \
           'blue_top_name', \
           'blue_jng_name', \
           'blue_mid_name', \
           'blue_bot_name', \
           'blue_sup_name',\
           'blue_team_result', \
           'red_team', \
           'red_top_name', \
           'red_jng_name', \
           'red_mid_name', \
           'red_bot_name', \
           'red_sup_name', \
           'blue_top_champ', \
           'blue_jng_champ', \
           'blue_mid_champ', \
           'blue_bot_champ', \
           'blue_sup_champ',\
           'red_top_champ', \
           'red_jng_champ', \
           'red_mid_champ', \
           'red_bot_champ', \
           'red_sup_champ']

lcs_rating = pd.DataFrame(columns=columns)

## Trueskill

In [9]:
ts = trueskill.TrueSkill(draw_probability=0.0)

In [10]:
player_ratings_dict = dict()
for i in input_data['player'].unique():
    player_ratings_dict[i] = ts.create_rating()
    
champ_ratings_dict = dict()
for i in input_data['champion'].unique():
    champ_ratings_dict[i] = ts.create_rating()

In [11]:
### Lambda Apply w Position as Key
def setup_match(df, gameid, position, champion):
    game_frame = df[df.gameid == gameid]
    blue_data = game_frame[game_frame.side == 'Blue']
    red_data = game_frame[game_frame.side == 'Red']
    #blue players
    blue_top = blue_data[blue_data.position == 'top']
    blue_top_name = blue_top.player.iloc[0]
    blue_top_champ = blue_top.champion.iloc[0]
    blue_jng = blue_data[blue_data.position == 'jng']
    blue_jng_name = blue_jng.player.iloc[0]
    blue_jng_champ = blue_jng.champion.iloc[0]
    blue_mid = blue_data[blue_data.position == 'mid']
    blue_mid_name = blue_mid.player.iloc[0]
    blue_mid_champ = blue_mid.champion.iloc[0]
    blue_bot = blue_data[blue_data.position == 'bot']
    blue_bot_name = blue_bot.player.iloc[0]
    blue_bot_champ = blue_bot.champion.iloc[0]
    blue_sup = blue_data[blue_data.position == 'sup']
    blue_sup_name = blue_sup.player.iloc[0]
    blue_sup_champ = blue_sup.champion.iloc[0]
    # red players
    red_top = red_data[red_data.position == 'top']
    red_top_name = red_top.player.iloc[0]
    red_top_champ = red_top.champion.iloc[0]
    red_jng = red_data[red_data.position == 'jng']
    red_jng_name = red_jng.player.iloc[0]
    red_jng_champ = red_jng.champion.iloc[0]
    red_mid = red_data[red_data.position == 'mid']
    red_mid_name = red_mid.player.iloc[0]
    red_mid_champ = red_mid.champion.iloc[0]
    red_bot = red_data[red_data.position == 'bot']
    red_bot_name = red_bot.player.iloc[0]
    red_bot_champ = red_bot.champion.iloc[0]
    red_sup = red_data[red_data.position == 'sup']
    red_sup_name = red_sup.player.iloc[0]
    red_sup_champ = red_sup.champion.iloc[0]
    # date, teams, result
    date = blue_data['date'].iloc[0]
    year = blue_data['year'].iloc[0]
    patch = blue_data['patch'].iloc[0]
    league = blue_data['league'].iloc[0]
    blue_team_result = blue_data['result'].iloc[0]
    blue_team = blue_data['team'].iloc[0]
    red_team = red_data['team'].iloc[0]
    # create pandas series for df
    match = pd.Series([gameid, date, year, patch, league, \
                       blue_team, \
                       blue_top_name, \
                       blue_jng_name, \
                       blue_mid_name, \
                       blue_bot_name, \
                       blue_sup_name,\
                       blue_team_result, \
                       red_team, \
                       red_top_name, \
                       red_jng_name, \
                       red_mid_name, \
                       red_bot_name, \
                       red_sup_name, \
                       blue_top_champ, \
                       blue_jng_champ, \
                       blue_mid_champ, \
                       blue_bot_champ, \
                       blue_sup_champ,\
                       red_top_champ, \
                       red_jng_champ, \
                       red_mid_champ, \
                       red_bot_champ, \
                       red_sup_champ])
    return match


lcs_rating[['gameid', 'date', 'year', 'patch', 'league', \
           'blue_team', \
           'blue_top_name', \
           'blue_jng_name', \
           'blue_mid_name', \
           'blue_bot_name', \
           'blue_sup_name',\
           'blue_team_result', \
           'red_team', \
           'red_top_name', \
           'red_jng_name', \
           'red_mid_name', \
           'red_bot_name', \
           'red_sup_name', \
           'blue_top_champ', \
           'blue_jng_champ', \
           'blue_mid_champ', \
           'blue_bot_champ', \
           'blue_sup_champ',\
           'red_top_champ', \
           'red_jng_champ', \
           'red_mid_champ', \
           'red_bot_champ', \
           'red_sup_champ']] = input_data.apply(lambda row: setup_match(input_data, row['gameid'], row['position'], row['champion']),axis=1)


In [12]:
lcs_rating.shape


(7590, 28)

In [13]:
lcs_rating = lcs_rating.reset_index(drop=True)
lcs_rating = lcs_rating.drop_duplicates(subset=['gameid'])
lcs_rating = lcs_rating.sort_values(by=['date'])

In [14]:
lcs_rating.head(20)

,gameid,date,year,patch,league,blue_team,blue_top_name,blue_jng_name,blue_mid_name,blue_bot_name,blue_sup_name,blue_team_result,red_team,red_top_name,red_jng_name,red_mid_name,red_bot_name,red_sup_name,blue_top_champ,blue_jng_champ,blue_mid_champ,blue_bot_champ,blue_sup_champ,red_top_champ,red_jng_champ,red_mid_champ,red_bot_champ,red_sup_champ
0,TRLH1/1002440062,2018-01-20 20:50:09,2018,8.01,NA LCS,Team Liquid,Impact,Xmithie,Pobelter,Doublelift,Olleh,1,TSM,Hauntzer,MikeYeung,Bjergsen,Zven,Mithy,Gangplank,Gragas,Malzahar,Tristana,Taric,Gnar,Shyvana,Ryze,Kalista,Alistar
10,TRLH1/1002440076,2018-01-20 22:38:20,2018,8.01,NA LCS,100 Thieves,Ssumday,Meteos,Ryu,Cody Sun,aphromoo,1,OpTic Gaming,zig,Akaadian,PowerOfEvil,Arrow,LemonNation,Shen,Sejuani,Ryze,Kog'Maw,Braum,Cho'Gath,Rengar,Malzahar,Ezreal,Morgana
20,TRLH1/1002440084,2018-01-21 00:14:20,2018,8.01,NA LCS,Clutch Gaming,Solo,Lira,FEBIVEN,Apollo,Hakuho,1,Golden Guardians,Lourlo,Contractz,Hai,Deftly,Matt,Gangplank,Evelynn,Malzahar,Tristana,Ornn,Vladimir,Kha'Zix,Azir,Kog'Maw,Taric
30,TRLH1/1002440095,2018-01-21 01:20:39,2018,8.01,NA LCS,Echo Fox,Huni,Dardoch,Fenix,Altec,Adrian,1,FlyQuest,Flame,AnDa,Keane,WildTurtle,Stunt,Lucian,Zac,Ryze,Kog'Maw,Shen,Gangplank,Jarvan IV,Cassiopeia,Varus,Alistar
40,TRLH1/1002440106,2018-01-21 02:16:56,2018,8.01,NA LCS,Counter Logic Gaming,ZionSpartan,Reignover,huhi,Stixxay,Biofrost,0,Cloud9,Licorice,Svenskeren,Jensen,Sneaky,Smoothie,Ornn,Rengar,Malzahar,Varus,Taric,Gangplank,Sejuani,Zilean,Tristana,Braum
50,TRLH1/1002440127,2018-01-21 21:49:32,2018,8.01,NA LCS,OpTic Gaming,zig,Akaadian,PowerOfEvil,Arrow,LemonNation,0,Team Liquid,Impact,Xmithie,Pobelter,Doublelift,Olleh,Ornn,Jarvan IV,Orianna,Kog'Maw,Taric,Gnar,Sejuani,Malzahar,Kalista,Thresh
60,TRLH1/1002440132,2018-01-21 22:46:16,2018,8.01,NA LCS,FlyQuest,Flame,AnDa,Keane,WildTurtle,Stunt,1,TSM,Hauntzer,MikeYeung,Bjergsen,Zven,Mithy,Gnar,Zac,Azir,Kalista,Taric,Cho'Gath,Kha'Zix,Xerath,Kog'Maw,Tahm Kench
70,TRLH1/1002440143,2018-01-22 00:00:28,2018,8.01,NA LCS,100 Thieves,Ssumday,Meteos,Ryu,Cody Sun,aphromoo,1,Counter Logic Gaming,ZionSpartan,Reignover,huhi,Stixxay,Biofrost,Gnar,Jarvan IV,Malzahar,Ezreal,Braum,Gangplank,Sejuani,Zilean,Caitlyn,Janna
80,TRLH1/1002440150,2018-01-22 01:00:51,2018,8.01,NA LCS,Golden Guardians,Lourlo,Contractz,Hai,Deftly,Matt,0,Cloud9,Licorice,Svenskeren,Jensen,Sneaky,Smoothie,Gangplank,Jarvan IV,Zoe,Varus,Taric,Kled,Evelynn,Ryze,Tristana,Tahm Kench
90,TRLH1/1002440161,2018-01-22 02:02:56,2018,8.01,NA LCS,Echo Fox,Huni,Dardoch,Fenix,Altec,Adrian,1,Clutch Gaming,Solo,Lira,FEBIVEN,Apollo,Hakuho,Gnar,Zac,Azir,Ezreal,Taric,Ornn,Sejuani,Ryze,Tristana,Braum


In [15]:
analyzed_gameids = {}

def win_probability(team1, team2, trueskill_global_env):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (trueskill_global_env.beta ** 2 ) + sum_sigma)
    return trueskill_global_env.cdf(delta_mu / denom)

def update_trueskill(player_rating_dict, champion_rating_dict, gameid_dict, gameid, blue_team_result, \
                        blue_top_name, \
                        blue_jng_name, \
                        blue_mid_name, \
                        blue_bot_name, \
                        blue_sup_name,\
                        red_top_name, \
                        red_jng_name, \
                        red_mid_name, \
                        red_bot_name, \
                        red_sup_name, \
                        blue_top_champ, \
                        blue_jng_champ, \
                        blue_mid_champ, \
                        blue_bot_champ, \
                        blue_sup_champ,\
                        red_top_champ, \
                        red_jng_champ, \
                        red_mid_champ, \
                        red_bot_champ, \
                        red_sup_champ):
    #####
    rating_groups = [(player_rating_dict[blue_top_name], \
                        player_rating_dict[blue_jng_name], \
                        player_rating_dict[blue_mid_name], \
                        player_rating_dict[blue_bot_name], \
                        player_rating_dict[blue_sup_name], \
                        champion_rating_dict[blue_top_champ], \
                        champion_rating_dict[blue_jng_champ], \
                        champion_rating_dict[blue_mid_champ], \
                        champion_rating_dict[blue_bot_champ], \
                        champion_rating_dict[blue_sup_champ]), \
                        (player_rating_dict[red_top_name], \
                        player_rating_dict[red_jng_name], \
                        player_rating_dict[red_mid_name], \
                        player_rating_dict[red_bot_name], \
                        player_rating_dict[red_sup_name], \
                        champion_rating_dict[red_top_champ], \
                        champion_rating_dict[red_jng_champ], \
                        champion_rating_dict[red_mid_champ], \
                        champion_rating_dict[red_bot_champ], \
                        champion_rating_dict[red_sup_champ])]
    blue_mu = rating_groups[0][0].mu
    blue_sigma = rating_groups[0][0].sigma
    red_mu = rating_groups[1][0].mu
    red_sigma = rating_groups[1][0].sigma
    blue_team_win_prob = win_probability(rating_groups[0], rating_groups[1], ts)
    # Update ratings
    if blue_team_result == 1:
        rated_rating_groups = ts.rate(rating_groups, ranks=[0, 1]) # for ranks 0 is winner 
    else:
        rated_rating_groups = ts.rate(rating_groups, ranks=[1, 0])
    # Return values for new columns
    ts_update = pd.Series([blue_team_win_prob, blue_mu, blue_sigma, red_mu, red_sigma])
    # update the rating dictionary
    # Blue Players
    player_rating_dict[blue_top_name] = rated_rating_groups[0][0]
    player_rating_dict[blue_jng_name] = rated_rating_groups[0][1]
    player_rating_dict[blue_mid_name] = rated_rating_groups[0][2]
    player_rating_dict[blue_bot_name] = rated_rating_groups[0][3]
    player_rating_dict[blue_sup_name] = rated_rating_groups[0][4]
    # Blue Champs
    champion_rating_dict[blue_top_champ] = rated_rating_groups[0][5]
    champion_rating_dict[blue_jng_champ] = rated_rating_groups[0][6]
    champion_rating_dict[blue_mid_champ] = rated_rating_groups[0][7]
    champion_rating_dict[blue_bot_champ] = rated_rating_groups[0][8]
    champion_rating_dict[blue_sup_champ] = rated_rating_groups[0][9]
    # Red Players
    player_rating_dict[red_top_name] = rated_rating_groups[1][0]
    player_rating_dict[red_jng_name] = rated_rating_groups[1][1]
    player_rating_dict[red_mid_name] = rated_rating_groups[1][2]
    player_rating_dict[red_bot_name] = rated_rating_groups[1][3]
    player_rating_dict[red_sup_name] = rated_rating_groups[1][4]
    # Red Champions
    champion_rating_dict[red_top_champ] = rated_rating_groups[1][5]
    champion_rating_dict[red_jng_champ] = rated_rating_groups[1][6]
    champion_rating_dict[red_mid_champ] = rated_rating_groups[1][7]
    champion_rating_dict[red_bot_champ] = rated_rating_groups[1][8]
    champion_rating_dict[red_sup_champ] = rated_rating_groups[1][9]
    # determine if use old data because of repeat
    if gameid in gameid_dict:
        return gameid_dict[gameid]
    else:
        gameid_dict[gameid] = ts_update
        return ts_update

lcs_rating[['blue_team_win_prob', 'blue_mu', 'blue_sigma', 'red_mu', 'red_sigma']] = lcs_rating.apply(lambda row: update_trueskill(player_ratings_dict, champ_ratings_dict, analyzed_gameids, row['gameid'], row['blue_team_result'], \
                        row['blue_top_name'], \
                        row['blue_jng_name'], \
                        row['blue_mid_name'], \
                        row['blue_bot_name'], \
                        row['blue_sup_name'],\
                        row['red_top_name'], \
                        row['red_jng_name'], \
                        row['red_mid_name'], \
                        row['red_bot_name'], \
                        row['red_sup_name'], \
                        row['blue_top_champ'], \
                        row['blue_jng_champ'], \
                        row['blue_mid_champ'], \
                        row['blue_bot_champ'], \
                        row['blue_sup_champ'],\
                        row['red_top_champ'], \
                        row['red_jng_champ'], \
                        row['red_mid_champ'], \
                        row['red_bot_champ'], \
                        row['red_sup_champ']), axis=1)

25.0
25.0
-----
26.329887365611572
26.329887365611572
-----
25.0
25.0
-----
25.0
25.0
-----
25.0
25.0
-----
25.0
25.0
-----
23.561375114413952
23.561375114413952
-----
23.655296279036016
23.655296279036016
-----
26.438624885586044
26.438624885586044
-----
23.69685020743197
23.69685020743197
-----
26.344703720963974
26.344703720963974
-----
24.733712241605453
24.733712241605453
-----
21.716883389303703
21.716883389303703
-----
27.04168403983148
27.04168403983148
-----
27.60766775991024
27.60766775991024
-----
22.82183710148927
22.82183710148927
-----
28.975859106388494
28.975859106388494
-----
25.97054509482131
25.97054509482131
-----
21.141477537510937
21.141477537510937
-----
21.98242263130832
21.98242263130832
-----
26.666618592859862
26.666618592859862
-----
22.397983475542702
22.397983475542702
-----
28.110922914430557
28.110922914430557
-----
24.900934492021015
24.900934492021015
-----
23.741115894189523
25.0
-----
22.649208982135512
22.649208982135512
-----
26.751919253107936
26.

32.39406352394973
-----
25.144511758681034
30.521445955476143
-----
27.51331678458134
25.11141230749401
-----
25.337540964286102
22.330174523766587
-----
26.015694219208818
23.155968811942188
-----
27.99512460874001
24.01197825069285
-----
29.147823797762182
25.43902045885626
-----
28.003701622309325
28.003701622309325
-----
29.3592013516735
29.3592013516735
-----
26.4310872452944
22.108519873474233
-----
26.65083528029595
23.975626833752766
-----
26.816846725974763
22.578117196165632
-----
25.62395271029016
22.63041154819145
-----
27.57900987131116
23.502228365834725
-----
30.247868207752514
30.247868207752514
-----
31.030261942152197
31.030261942152197
-----
31.33706799832702
31.33706799832702
-----
31.56733118962281
31.56733118962281
-----
27.085214965414927
22.909765462835892
-----
22.595738968723694
24.291690742129894
-----
25.0
26.486722879410756
-----
28.06442091357796
22.430163226592637
-----
26.11000892344674
27.084307988757224
-----
24.867432328996923
22.778498848816117
-----

-----
28.3142755861777
28.234471405544653
-----
32.83967473586103
32.83967473586103
-----
27.46662550943249
28.01923530470655
-----
32.28416527491426
34.246415450901004
-----
22.92616076212497
24.84782848722318
-----
25.0
33.226738528747966
-----
30.039520404068057
26.722254910187594
-----
31.038830659664963
26.994689658641242
-----
27.054048706982428
24.503170607104877
-----
21.9611167772352
23.35143557862348
-----
26.410301985433843
27.030042006850678
-----
32.87360702893143
23.30659380603029
-----
30.739366595854356
33.02392688908708
-----
30.25242388253984
27.108571604071432
-----
27.942286247228157
29.238131277050915
-----
28.662564672706434
27.737534969122567
-----
33.096229381523074
24.41288829514493
-----
25.842881996017375
26.824169924099284
-----
22.931296452609416
24.852732903936825
-----
23.111135351541616
32.838712804456414
-----
22.2957866360124
23.619323595708796
-----
30.224545297379517
26.055006060211728
-----
24.79626085995798
26.475036004242394
-----
22.8065885855231

22.932749979621356
22.474297492077397
-----
29.139025751152776
27.21901672725273
-----
22.799473333902096
21.969598820655538
-----
33.76864047753773
28.499623889866818
-----
23.552096472326454
26.34891106663659
-----
22.769502674504025
21.858301361104875
-----
30.779591614366506
24.24538200209391
-----
29.276923331653403
31.73414176733273
-----
23.67394065762066
26.461504905589038
-----
28.440276180756907
26.56789411732001
-----
24.136778515564718
26.88956969725932
-----
23.485277977838592
26.286363168231144
-----
22.531647535941616
28.22019751717296
-----
22.308289858409793
28.020812539580817
-----
30.466025710117126
23.396670231546768
-----
28.677062483089625
26.78892552622393
-----
23.008329916792153
21.056331627296327
-----
28.991612264448232
27.083106747150037
-----
28.52397492388684
31.47386926188216
-----
28.093283356401574
30.710219827175273
-----
28.387346160409137
31.226895222020012
-----
34.09237687498293
29.36672375800421
-----
34.32722692454684
29.985048305643744
-----
34.

In [16]:
def predict_blue_win(win_prob):
    if win_prob > .5:
        return 1
    else:
        return 0

lcs_rating['blue_predicted_to_win'] = lcs_rating['blue_team_win_prob'].apply(lambda x: predict_blue_win(x))
    
lcs_rating.shape

(759, 34)

In [17]:
lcs_rating = lcs_rating.reset_index(drop=True)
lcs_rating.head(20)

,gameid,date,year,patch,league,blue_team,blue_top_name,blue_jng_name,blue_mid_name,blue_bot_name,blue_sup_name,blue_team_result,red_team,red_top_name,red_jng_name,red_mid_name,red_bot_name,red_sup_name,blue_top_champ,blue_jng_champ,blue_mid_champ,blue_bot_champ,blue_sup_champ,red_top_champ,red_jng_champ,red_mid_champ,red_bot_champ,red_sup_champ,blue_team_win_prob,blue_mu,blue_sigma,red_mu,red_sigma,blue_predicted_to_win
0,TRLH1/1002440062,2018-01-20 20:50:09,2018,8.01,NA LCS,Team Liquid,Impact,Xmithie,Pobelter,Doublelift,Olleh,1,TSM,Hauntzer,MikeYeung,Bjergsen,Zven,Mithy,Gangplank,Gragas,Malzahar,Tristana,Taric,Gnar,Shyvana,Ryze,Kalista,Alistar,0.500000,25.000000,8.333333,25.000000,8.333333,1
1,TRLH1/1002440076,2018-01-20 22:38:20,2018,8.01,NA LCS,100 Thieves,Ssumday,Meteos,Ryu,Cody Sun,aphromoo,1,OpTic Gaming,zig,Akaadian,PowerOfEvil,Arrow,LemonNation,Shen,Sejuani,Ryze,Kog'Maw,Braum,Cho'Gath,Rengar,Malzahar,Ezreal,Morgana,0.460776,25.000000,8.333333,25.000000,8.333333,0
2,TRLH1/1002440084,2018-01-21 00:14:20,2018,8.01,NA LCS,Clutch Gaming,Solo,Lira,FEBIVEN,Apollo,Hakuho,1,Golden Guardians,Lourlo,Contractz,Hai,Deftly,Matt,Gangplank,Evelynn,Malzahar,Tristana,Ornn,Vladimir,Kha'Zix,Azir,Kog'Maw,Taric,0.512325,25.000000,8.333333,25.000000,8.333333,1
3,TRLH1/1002440095,2018-01-21 01:20:39,2018,8.01,NA LCS,Echo Fox,Huni,Dardoch,Fenix,Altec,Adrian,1,FlyQuest,Flame,AnDa,Keane,WildTurtle,Stunt,Lucian,Zac,Ryze,Kog'Maw,Shen,Gangplank,Jarvan IV,Cassiopeia,Varus,Alistar,0.497015,25.000000,8.333333,25.000000,8.333333,0
4,TRLH1/1002440106,2018-01-21 02:16:56,2018,8.01,NA LCS,Counter Logic Gaming,ZionSpartan,Reignover,huhi,Stixxay,Biofrost,0,Cloud9,Licorice,Svenskeren,Jensen,Sneaky,Smoothie,Ornn,Rengar,Malzahar,Varus,Taric,Gangplank,Sejuani,Zilean,Tristana,Braum,0.432677,25.000000,8.333333,25.000000,8.333333,0
5,TRLH1/1002440127,2018-01-21 21:49:32,2018,8.01,NA LCS,OpTic Gaming,zig,Akaadian,PowerOfEvil,Arrow,LemonNation,0,Team Liquid,Impact,Xmithie,Pobelter,Doublelift,Olleh,Ornn,Jarvan IV,Orianna,Kog'Maw,Taric,Gnar,Sejuani,Malzahar,Kalista,Thresh,0.341286,23.561375,8.223034,27.048081,8.149465,0
6,TRLH1/1002440132,2018-01-21 22:46:16,2018,8.01,NA LCS,FlyQuest,Flame,AnDa,Keane,WildTurtle,Stunt,1,TSM,Hauntzer,MikeYeung,Bjergsen,Zven,Mithy,Gnar,Zac,Azir,Kalista,Taric,Cho'Gath,Kha'Zix,Xerath,Kog'Maw,Tahm Kench,0.522273,23.655296,8.225571,22.951919,8.149465,1
7,TRLH1/1002440143,2018-01-22 00:00:28,2018,8.01,NA LCS,100 Thieves,Ssumday,Meteos,Ryu,Cody Sun,aphromoo,1,Counter Logic Gaming,ZionSpartan,Reignover,huhi,Stixxay,Biofrost,Gnar,Jarvan IV,Malzahar,Ezreal,Braum,Gangplank,Sejuani,Zilean,Caitlyn,Janna,0.557056,26.438625,8.223034,23.834290,8.231653,1
8,TRLH1/1002440150,2018-01-22 01:00:51,2018,8.01,NA LCS,Golden Guardians,Lourlo,Contractz,Hai,Deftly,Matt,0,Cloud9,Licorice,Svenskeren,Jensen,Sneaky,Smoothie,Gangplank,Jarvan IV,Zoe,Varus,Taric,Kled,Evelynn,Ryze,Tristana,Tahm Kench,0.322371,23.696850,8.227136,26.165710,8.231653,0
9,TRLH1/1002440161,2018-01-22 02:02:56,2018,8.01,NA LCS,Echo Fox,Huni,Dardoch,Fenix,Altec,Adrian,1,Clutch Gaming,Solo,Lira,FEBIVEN,Apollo,Hakuho,Gnar,Zac,Azir,Ezreal,Taric,Ornn,Sejuani,Ryze,Tristana,Braum,0.412038,26.344704,8.225571,26.303150,8.227136,0


### Check Accuracy

In [18]:
# All Data
y_true = lcs_rating['blue_team_result']
y_pred_bool = lcs_rating['blue_predicted_to_win']
y_pred_pct = lcs_rating['blue_team_win_prob']

print('---Metrics for all data---')
print('F1 Score: ', f1_score(y_true, y_pred_bool))
print('Precision Score: ', precision_score(y_true, y_pred_bool))
print('Recall Score: ', recall_score(y_true, y_pred_bool))
print('-')
print('Log Loss: ', log_loss(y_true, y_pred_pct))
print()
# Primed on 2018-2019, Results Measured on 2020-2021
lcs_rating_2020on = lcs_rating[lcs_rating['year'] >= 2020]
y_true = lcs_rating_2020on['blue_team_result']
y_pred_bool = lcs_rating_2020on['blue_predicted_to_win']
y_pred_pct = lcs_rating_2020on['blue_team_win_prob']

print('---Metrics for 2020-2021 data---')
print('F1 Score: ', f1_score(y_true, y_pred_bool))
print('Precision Score: ', precision_score(y_true, y_pred_bool))
print('Recall Score: ', recall_score(y_true, y_pred_bool))
print('-')
print('Log Loss: ', log_loss(y_true, y_pred_pct))


---Metrics for all data---
F1 Score:  0.6420047732696896
Precision Score:  0.6810126582278481
Recall Score:  0.6072234762979684
-
Log Loss:  0.7173251317275251

---Metrics for 2020-2021 data---
F1 Score:  0.6585365853658536
Precision Score:  0.6835443037974683
Recall Score:  0.6352941176470588
-
Log Loss:  0.7166020515322337


### Average Rating Change for a Player on a given Champion

In [19]:
champion_rating_change_dict = dict()
for i in input_data['champion'].unique():
    champion_rating_change_dict[i] = []
    
for index, row in lcs_rating.iterrows():
    champion_rating_change_dict[row['blue_top_champ']].append(row['new_blue_top_mu'] - row['blue_top_mu'])
    champion_rating_change_dict[row['blue_jng_champ']].append(row['new_blue_jng_mu'] - row['blue_jng_mu'])
    champion_rating_change_dict[row['blue_mid_champ']].append(row['new_blue_mid_mu'] - row['blue_mid_mu'])
    champion_rating_change_dict[row['blue_bot_champ']].append(row['new_blue_bot_mu'] - row['blue_bot_mu'])
    champion_rating_change_dict[row['blue_sup_champ']].append(row['new_blue_sup_mu'] - row['blue_sup_mu'])
    champion_rating_change_dict[row['red_top_champ']].append(row['new_red_top_mu'] - row['red_top_mu'])
    champion_rating_change_dict[row['red_jng_champ']].append(row['new_red_jng_mu'] - row['red_jng_mu'])
    champion_rating_change_dict[row['red_mid_champ']].append(row['new_red_mid_mu'] - row['red_mid_mu'])
    champion_rating_change_dict[row['red_bot_champ']].append(row['new_red_bot_mu'] - row['red_bot_mu'])
    champion_rating_change_dict[row['red_sup_champ']].append(row['new_red_sup_mu'] - row['red_sup_mu'])

for champ in champion_rating_change_dict:
    champion_rating_change_dict[champ] = sum(champion_rating_change_dict[champ]) / len(champion_rating_change_dict[champ])

champ_json = json.dumps(champion_rating_change_dict, sort_keys=True)

print(champ_json)


KeyError: 'new_blue_top_mu'

### Calculate Future Winner from Input Teams

Team1 and Team2 input tuples for win_probability function  

(trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333))  

(trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333), trueskill.Rating(mu=25.000, sigma=8.333))

In [ ]:
#print(player_ratings_dict)

In [ ]:
TSM = ['Huni', 'Spica', 'PowerOfEvil', 'Lost', 'SwordArt']
FLY = ['Licorice', 'Josedeodo', 'Palafox', 'Johnsun', 'Diamond']
TSM_RATINGS = []
FLY_RATINGS = []

for player in TSM:
    TSM_RATINGS.append(player_ratings_dict[player])

for player in FLY:
    FLY_RATINGS.append(player_ratings_dict[player])
    
TSM_TUPLE = tuple(TSM_RATINGS)
FLY_TUPLE = tuple(FLY_RATINGS)

team_1_win_prob = win_probability(TSM_TUPLE, FLY_TUPLE, ts)
print('TSM Win Probability :' + str(team_1_win_prob))

In [ ]:
#print(analyzed_gameids)

In [ ]:
#lcs_rating.to_csv('./lcs_player_x_champion_trueskill_2018-2021.csv', index=False)